In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('/content/features_30_sec.csv')

# Separate features and target variable
X = data.drop(['label', 'filename'], axis=1)
y = data['label']

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded)

# Define hyperparameters to tune
hyperparameters = {
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'conv1_filters': [64, 128, 256],
    'dense_units': [32, 64, 128]
}

def create_model(conv1_filters, dense_units):
    model = Sequential([
        Conv1D(conv1_filters, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(2),
        Flatten(),
        Dense(dense_units, activation='relu'),
        Dropout(0.5),
        Dense(y_one_hot.shape[1], activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_hyperparameters = None
best_accuracy = 0

for batch_size in hyperparameters['batch_size']:
    for epochs in hyperparameters['epochs']:
        for conv1_filters in hyperparameters['conv1_filters']:
            for dense_units in hyperparameters['dense_units']:
                accuracies = []
                for train_index, val_index in skf.split(X, y_encoded):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y_one_hot[train_index], y_one_hot[val_index]

                    scaler = StandardScaler()
                    X_train_scaled = scaler.fit_transform(X_train)
                    X_val_scaled = scaler.transform(X_val)

                    X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
                    X_val_scaled = X_val_scaled.reshape(X_val_scaled.shape[0], X_val_scaled.shape[1], 1)

                    model = create_model(conv1_filters, dense_units)

                    history = model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val_scaled, y_val), verbose=0)

                    y_val_pred = model.predict(X_val_scaled)
                    y_val_pred_classes = np.argmax(y_val_pred, axis=1)
                    y_val_true_classes = np.argmax(y_val, axis=1)

                    accuracy = accuracy_score(y_val_true_classes, y_val_pred_classes)
                    accuracies.append(accuracy)

                mean_accuracy = np.mean(accuracies)
                if mean_accuracy > best_accuracy:
                    best_accuracy = mean_accuracy
                    best_hyperparameters = (batch_size, epochs, conv1_filters, dense_units)

print("Best Hyperparameters:", best_hyperparameters)
print("Best Accuracy:", best_accuracy)

7/7 [==============================] - 0s 4ms/step
Best Hyperparameters: (32, 30, 256, 128)
Best Accuracy: 0.743
